<a href="https://colab.research.google.com/github/C22-PS008/machine-learning/blob/main/train/dialog-gpt-based/dialog_gpt_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tokenizers
!pip install datasets
!pip install sentencepiece
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 62.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 12.1 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 140 kB 75.4 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 62.5 MB/s 
     |██████████████████████████████

In [ ]:
from transformers import GPT2Tokenizer, TFGPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# load dataset
blended_skill_dataset = load_dataset("blended_skill_talk")

# split the dataset
blended_skill_dataset_train = blended_skill_dataset['train']
blended_skill_dataset_validation = blended_skill_dataset['validation']
blended_skill_dataset_test = blended_skill_dataset['test']

Using custom data configuration default


Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

Dataset blended_skill_talk downloaded and prepared to /root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
previous_utterance_total=blended_skill_dataset_train['previous_utterance']+ blended_skill_dataset_validation['previous_utterance']+blended_skill_dataset_test['previous_utterance']
free_messages_total=blended_skill_dataset_train['free_messages']+ blended_skill_dataset_validation['free_messages']+blended_skill_dataset_test['free_messages']
guided_messages_total=blended_skill_dataset_train['guided_messages']+ blended_skill_dataset_validation['guided_messages']+blended_skill_dataset_test['guided_messages']

In [ ]:
# define punctutation
space = 'Ġ'
pre_quote = '’'
end_marks = ['.', ',', '?', '!', '...']
quotes = ['"', '\'']
abbreviations = ['s', 'd', 't', 'm', 're', 'll', 've', 'S', 'D', 'T', 'M', 'Re', 'Ll', 'Ve']

In [ ]:
from datasets import *

def process_token_list(token_list):
    token_list[0] = token_list[0].capitalize()
    
    quote_count = 0
    for i, token in enumerate(token_list):

        if space in token:
            if token[1:] in end_marks or token[1:] in abbreviations:
                token_list[i] = token[1:]
                
            if token[1:] == quotes[1]:
                if i < len(token_list)-1:
                    if token_list[i + 1] in abbreviations or (token_list[i + 1][0] == space and token_list[i + 1][1:] in abbreviations):
                        token_list[i] = token[1:]
                        
        if token[0] == space and token[1:] in quotes:
            if quote_count % 2 == 1:
                token_list[i] = token[1:]
                quote_count = 0
            else:
                if i < len(token_list)-1 and token_list[i + 1][0] == space:
                    token_list[i + 1] = token_list[i + 1][1:]
                quote_count += 1
                
        if token in end_marks or token[1:] in end_marks:
            if i<len(token_list)-1:
                if token_list[i + 1][0] != space:
                    token_list[i + 1] = space + token_list[i + 1].capitalize()
                else:
                    token_list[i + 1] = space + token_list[i + 1][1:].capitalize()
                
    new_token_list = [token for token in token_list if token != space and len(token) > 0]
    if new_token_list[-1] not in end_marks:
        new_token_list.append(end_marks[0])
        
    return new_token_list

In [ ]:
from tqdm import tqdm

def load_blended(tokenizer, train_frac):
  total_dialogues=[]
  for i, free_message in enumerate(tqdm(free_messages_total)):
      free_messages_list = [utterance.strip() for utterance in free_message if len(utterance.strip())>0]
      guided_messages_list = [utterance.strip() for utterance in guided_messages_total[i] if len(utterance.strip())>0]
      dialogue=previous_utterance_total[i]
      
      for j in range(len(free_messages_list)):
          token_list = process_token_list(tokenizer.tokenize(free_messages_list[j]))
          text=tokenizer.convert_tokens_to_string(token_list)
          total_dialogues.append(text)

          if j<len(guided_messages_list):
              token_list = process_token_list(tokenizer.tokenize(guided_messages_list[j]))
              text=tokenizer.convert_tokens_to_string(token_list)
              total_dialogues.append(text)
      total_dialogues.append(dialogue)

  train_utter_num = 0
  valid_utter_num = 0
  train_dialogues = total_dialogues[:int(len(total_dialogues)*train_frac)]
  valid_dialogues = total_dialogues[int(len(total_dialogues)*train_frac):]
  
  for dialogue in train_dialogues:
      train_utter_num += len(dialogue)
      
  for dialogue in valid_dialogues:
      valid_utter_num += len(dialogue)
  
  return train_dialogues, valid_dialogues, train_utter_num, valid_utter_num

In [ ]:
def merge_data(tokenizer, train_frac):
    train_dialogues = []
    valid_dialogues = []
    num_train = 0
    num_valid = 0
    for data_name in blended_skill_dataset:
        part_train_dialogues, part_valid_dialogues, part_num_train, part_num_valid = load_blended(tokenizer, train_frac)
        
        train_dialogues += part_train_dialogues
        valid_dialogues += part_valid_dialogues
    
        print("#"*50 + f"Analysis on {data_name}" + "#"*50)
        print(f"The number of train dialogues: {len(part_train_dialogues)}")
        print(f"The number of valid dialogues: {len(part_valid_dialogues)}")    
        print(f"The number of train utterances: {part_num_train}")    
        print(f"The number of valid utterances: {part_num_valid}")
        
        num_train += part_num_train
        num_valid += part_num_valid
        
    return train_dialogues, valid_dialogues, num_train, num_valid

In [ ]:
def tokenize_dataset(tokenizer, split):
  total_dialogues=[]
  for i, free_message in enumerate(tqdm(free_messages_total)):
      free_messages_list = [utterance.strip() for utterance in free_message if len(utterance.strip())>0]
      guided_messages_list = [utterance.strip() for utterance in guided_messages_total[i] if len(utterance.strip())>0]
      dialogue=previous_utterance_total[i]
      
      for j in range(len(free_messages_list)):
          token_list = process_token_list(tokenizer.tokenize(free_messages_list[j]))
          text=tokenizer.convert_tokens_to_string(token_list)
          total_dialogues.append(text)

          if j<len(guided_messages_list):
              token_list = process_token_list(tokenizer.tokenize(guided_messages_list[j]))
              text=tokenizer.convert_tokens_to_string(token_list)
              total_dialogues.append(text)
      total_dialogues.append(dialogue)

  train_utter_num = 0
  valid_utter_num = 0
  train_dialogues = total_dialogues[:int(len(total_dialogues)*split)]
  valid_dialogues = total_dialogues[int(len(total_dialogues)*split):]
  
  for dialogue in train_dialogues:
      train_utter_num += len(dialogue)
      
  for dialogue in valid_dialogues:
      valid_utter_num += len(dialogue)
  train_ids = []
  for dialogue in train_dialogues:
    dialogue_ids = []
    for utter in dialogue:
        tokens = tokenizer.tokenize(utter)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        dialogue_ids.append(token_ids)
    ids.append(dialogue_ids)

  assert len(train_ids) == len(train_dialogues)
train_ids = []
for dialogue in train_dialogues:
  dialogue_ids = []
  for utter in dialogue:
      tokens = tokenizer.tokenize(utter)
      token_ids = tokenizer.convert_tokens_to_ids(tokens)
      dialogue_ids.append(token_ids)
  ids.append(dialogue_ids)

assert len(train_ids) == len(train_dialogues)

In [ ]:
from torch.utils.data import DataLoader

torch_train_dataset = DataLoader(tokenize_indonlu_dataset["train"],shuffle=True,collate_fn=pytorch_data_collator,
    batch_size=16,
)
torch_validation_dataset = DataLoader(
    tokenize_indonlu_dataset["validation"], collate_fn=pytorch_data_collator, batch_size=16
)

In [ ]:
import json

def save_data(prefix, data_dir, dialogues, tokenizer):
    print(f"Saving {prefix} text file...")
    with open(f"{data_dir}/{prefix}_utters.json", 'w') as f:
        json.dump(dialogues, f)
    
    print(f"Saving {prefix} idx file...")
    ids = []
    for dialogue in tqdm(dialogues):
        dialogue_ids = []
        for utter in dialogue:
            tokens = tokenizer.tokenize(utter)
            token_ids = tokenizer.convert_tokens_to_ids(tokens)
            dialogue_ids.append(token_ids)
        ids.append(dialogue_ids)
        
    assert len(ids) == len(dialogues)
        
    with open(f"{data_dir}/{prefix}_ids.json", 'w') as f:
        json.dump(ids, f)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [ ]:
# file location
import os

dir = os.getcwd()
dataset_dir=dir+'/data'
dataset_dir

In [ ]:
!mkdir /content/data

In [ ]:
train_frac = 0.85

train_dialogues, valid_dialogues, num_train, num_valid = merge_data(tokenizer, train_frac)
print("Saving train data...")
save_data('train', dir, train_dialogues, tokenizer)
print("Saving validation data...")
save_data('valid', dir, valid_dialogues, tokenizer)

100%|██████████| 6808/6808 [00:12<00:00, 530.18it/s]


##################################################Analysis on train##################################################
The number of train dialogues: 70642
The number of valid dialogues: 12467
The number of train utterances: 4384260
The number of valid utterances: 800754


100%|██████████| 6808/6808 [00:12<00:00, 527.64it/s]


##################################################Analysis on validation##################################################
The number of train dialogues: 70642
The number of valid dialogues: 12467
The number of train utterances: 4384260
The number of valid utterances: 800754


100%|██████████| 6808/6808 [00:12<00:00, 529.23it/s]


##################################################Analysis on test##################################################
The number of train dialogues: 70642
The number of valid dialogues: 12467
The number of train utterances: 4384260
The number of valid utterances: 800754
Saving train data...
Saving train text file...
Saving train idx file...


100%|██████████| 211926/211926 [27:32<00:00, 128.26it/s]


Saving validation data...
Saving valid text file...
Saving valid idx file...


100%|██████████| 37401/37401 [13:32<00:00, 46.01it/s]


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer",push_to_hub=True,)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ids,
    eval_dataset=valid_dialogues,
)

Cloning https://huggingface.co/chanifrusydi/test_trainer into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 211926
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 79473


TypeError: ignored

In [ ]:
trainer.push_to_hub()